<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="https://www.uoc.edu/content/dam/news/images/noticies/2016/202-nova-marca-uoc.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.879 · TFM - Área 2 - Aula 1</p>
<p style="margin: 0; text-align:right;">2023-2 · Máster universitario en Ciencia de datos (<i>Data science</i>)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# **Carga, preprocesamiento, aumento y codificación de imágenes en impulsos (Spikes)**

## Introducción

El presente código está diseñado para transformar un conjunto de datos de imágenes de conducción autónoma en un formato adecuado para ser utilizado en redes neuronales de impulsos (SNN), específicamente codificando las imágenes en impulsos utilizando la técnica de modulación delta de la librería [snnTorch](https://snntorch.readthedocs.io/en/latest/index.html). Este proceso es fundamental para entrenar una red neuronal convolucional de impulsos (CSNN) que pueda emular el comportamiento de redes neuronales biológicas, aprovechando su eficiencia energética y su capacidad para procesar información temporal.

El código se estructura en varias etapas que abarcan desde la carga y preprocesamiento de datos, hasta la codificación de las imágenes y la medición de la eficiencia energética del proceso. A continuación se detallan las principales etapas y funcionalidades del código:

1. **Montar Google Drive y Descomprimir el Dataset**: Se monta Google Drive para acceder a los datos y se descomprime el archivo del dataset de entrenamiento.
   
2. **Instalación de Librerías**: Se instalan las librerías necesarias para el procesamiento y codificación de imágenes, así como para el seguimiento de emisiones de carbono.

3. **Importación de Librerías**: Se importan las librerías necesarias para la manipulación de datos, procesamiento de imágenes, codificación en impulsos y seguimiento de emisiones de carbono.

4. **Funciones de Carga y Preprocesamiento de Datos**: Se definen funciones para cargar los datos de conducción desde archivos CSV, simplificar rutas de imágenes, categorizar ángulos de giro, equilibrar el dataset y dividirlo en conjuntos de entrenamiento y validación.

5. **Funciones de Aumento de Datos**: Se implementan diversas técnicas de aumento de datos, como zoom, volteo, pan, brillo y sombras aleatorias, para aumentar la variedad del dataset y mejorar el entrenamiento del modelo.

6. **Funciones de Preprocesamiento de Imágenes y Codificación de Impulsos**: Se preprocesan las imágenes para convertirlas en un formato adecuado para la codificación en impulsos, normalizan los ángulos de dirección y realizan la codificación delta utilizando la librería `snntorch`.

7. **Carga y Balanceo de Datos de Conducción**: Se carga y balancea el dataset de conducción para crear conjuntos de entrenamiento y validación.

8. **Codificación de Datos y Guardado de Resultados**: Se inicia el tracker de CodeCarbon para medir las emisiones de carbono durante el proceso de codificación, se preparan y codifican los datos de entrenamiento y validación, y se guardan los resultados en archivos HDF5. Finalmente, se copian los archivos a Google Drive.

Este enfoque permite no solo la preparación y transformación de los datos en un formato adecuado para SNN, sino también la evaluación de la eficiencia energética del proceso, lo cual es crucial para desarrollar soluciones sostenibles en el campo de la conducción autónoma.

1. **Montar Google Drive y Descomprimir el Dataset**: Montamos Google Drive y descomprimimos el dataset que contiene las imágenes de entrenamiento.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!unzip "/content/drive/MyDrive/UOC/TFM/Dataset/TrainImagesTFM_1_2.zip" -d "/content"

2. **Instalación de Librerías**: Instalamos las librerías necesarias para el procesamiento y codificación de las imágenes, así como para medir la eficiencia energética.

In [3]:
pip install codecarbon torch snntorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.1/496.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 27.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl

3. **Importación de Librerías**: Importamos las librerías necesarias para manipulación de datos, procesamiento de imágenes, codificación de impulsos y seguimiento de emisiones de carbono.

In [4]:
import torch
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
from imgaug import augmenters as iaa
import os
import h5py
import snntorch.spikegen as spikegen
from tqdm import tqdm
from codecarbon import EmissionsTracker
import cv2
import matplotlib.image as mpimg
from datetime import datetime
import csv
import shutil

4. **Funciones de Carga y Preprocesamiento de Datos**: Estas funciones se utilizan para cargar y preprocesar los datos de conducción desde archivos CSV, equilibrar el dataset y dividirlo en conjuntos de entrenamiento y validación.

In [5]:
# Cargar datos de conducción desde un archivo CSV
def load_driving_data(csv_file_path):
    data = pd.read_csv(csv_file_path)
    data.columns = [
        'center_image', 'left_image', 'right_image',
        'steering_angle', 'throttle', 'brake', 'speed'
    ]
    return data

# Simplificar las rutas de las imágenes en el DataFrame
def simplify_image_path(dataframe, columns):
    for column in columns:
        dataframe[column] = dataframe[column].apply(lambda x: x.split('/')[-1])
    return dataframe

# Categorizar los ángulos de giro en el DataFrame
def categorize_turns(data, angle_column):
    bins = [-float('inf'), -0.1, 0.1, float('inf')]
    labels = ['Left Turn', 'Straight', 'Right Turn']
    data['turn_category'] = pd.cut(data[angle_column], bins=bins, labels=labels)
    turn_counts = data['turn_category'].value_counts()
    return turn_counts

# Equilibrar los datos replicando registros en categorías menos representadas
def balance_data_by_replication(data, category_column, target_size=None):
    categories = data[category_column].unique()
    subset_list = [data[data[category_column] == category] for category in categories]
    if not target_size:
        target_size = max(subset.shape[0] for subset in subset_list)
    balanced_subsets = [subset.sample(target_size, replace=True) if subset.shape[0] < target_size else subset for subset in subset_list]
    balanced_data = pd.concat(balanced_subsets, ignore_index=True)
    return balanced_data

# Calcular el número máximo de muestras por bin basado en un percentil dado
def calculate_samples_per_bin(df, steering_col, num_bins, percentile):
    hist, _ = np.histogram(df[steering_col], bins=num_bins)
    max_samples = np.percentile(hist, percentile)
    return int(max_samples)

# Ajustar la distribución de los ángulos de dirección para que sea más uniforme por categoría de giro
def remove_samples_by_category(df, category_col='turn_category', steering_col='steering_angle', num_bins=25, samples_per_bin=None):
    df.reset_index(drop=True, inplace=True)
    categories = df[category_col].unique()
    balanced_dfs = []

    for category in categories:
        category_df = df[df[category_col] == category]
        hist, bins = np.histogram(category_df[steering_col], bins=num_bins)
        if samples_per_bin is None:
            samples_per_bin = int(np.percentile(hist, 90))
        remove_list = []

        for j in range(num_bins):
            list_indices = [i for i in range(len(category_df[steering_col])) if bins[j] <= category_df[steering_col].iloc[i] < bins[j + 1]]
            list_indices = shuffle(list_indices)
            if len(list_indices) > samples_per_bin:
                remove_list.extend(list_indices[samples_per_bin:])

        category_df = category_df.drop(category_df.index[remove_list])
        balanced_dfs.append(category_df)

    return pd.concat(balanced_dfs, ignore_index=True)

# Asocia cada imagen de las cámaras central, izquierda y derecha con un ángulo de giro ajustado
def load_img_steering(datadir, df):
    image_path = []
    steering = []
    for i in range(len(df)):
        indexed_data = df.iloc[i]
        center, left, right = indexed_data[0], indexed_data[1], indexed_data[2]
        image_path.append(os.path.join(datadir, center.strip()))
        steering.append(float(indexed_data[3]))
        image_path.append(os.path.join(datadir, left.strip()))
        steering.append(float(indexed_data[3]) + 0.2)
        image_path.append(os.path.join(datadir, right.strip()))
        steering.append(float(indexed_data[3]) - 0.2)
    return np.array(image_path), np.array(steering)

# Crea datos de entrenamiento y validación
def trainValSample(dir, df):
    if not dir.endswith("/"):
        dir += "/"
    image_paths, steerings = load_img_steering(dir + "IMG/", df)
    X_train, X_valid, y_train, y_valid = train_test_split(image_paths, steerings, test_size=0.2, random_state=42)
    print("Training Samples: {}\nValidation Samples: {}".format(len(X_train), len(X_valid)))
    print("Total Samples: {}".format(len(image_paths)))
    return image_paths, steerings, X_train, X_valid, y_train, y_valid

5. **Funciones de Aumento de Datos**: Estas funciones aplican diversas técnicas de aumento de datos, como zoom, volteo, pan, brillo y sombras aleatorias, para aumentar la variedad del dataset y mejorar el entrenamiento del modelo.

In [6]:
# Función para convertir la imagen a escala de grises
def convert_to_grayscale(image):
    if image is None or image.size == 0:
        raise ValueError("Empty image provided for grayscale conversion.")
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Funciones de aumento de imagen
def apply_zoom(image):
    zoom_transformer = iaa.Affine(scale=(1, 1.5))
    zoomed_image = zoom_transformer.augment_image(image)
    return zoomed_image

def random_flip(image, steering_angle):
    flipped_image = cv2.flip(image, 1)
    flipped_steering_angle = -steering_angle
    return flipped_image, flipped_steering_angle

def apply_pan(image):
    pan_transformer = iaa.Affine(translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)})
    panned_image = pan_transformer.augment_image(image)
    return panned_image

def apply_random_brightness(image):
    brightness_augmenter = iaa.Multiply((0.2, 1.6))
    brightened_image = brightness_augmenter.augment_image(image)
    return brightened_image

def apply_image_brighten(image):
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)  # Convertir a BGR para trabajar en el espacio HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    brightness = 0.25 + np.random.uniform()
    hsv_image[:, :, 2] = hsv_image[:, :, 2] * brightness
    hsv_image[:, :, 2] = np.clip(hsv_image[:, :, 2], 0, 255)
    brightened_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
    brightened_image = cv2.cvtColor(brightened_image, cv2.COLOR_BGR2GRAY)  # Convertir de vuelta a escala de grises
    return brightened_image

def apply_random_shadow(image):
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)  # Convertir a BGR para trabajar en el espacio HLS
    IMAGE_HEIGHT, IMAGE_WIDTH = image.shape[:2]
    x1, y1 = IMAGE_WIDTH * np.random.rand(), 0
    x2, y2 = IMAGE_WIDTH * np.random.rand(), IMAGE_HEIGHT
    xm, ym = np.mgrid[0:IMAGE_HEIGHT, 0:IMAGE_WIDTH]
    mask = np.zeros_like(image[:, :, 1])
    mask[((ym - y1) * (x2 - x1) - (y2 - y1) * (xm - x1)) > 0] = 1
    cond = mask == np.random.randint(2)
    s_ratio = np.random.uniform(low=0.2, high=0.5)
    hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    hls[:, :, 1][cond] = hls[:, :, 1][cond] * s_ratio
    shadowed_image = cv2.cvtColor(hls, cv2.COLOR_HLS2BGR)
    shadowed_image = cv2.cvtColor(shadowed_image, cv2.COLOR_BGR2GRAY)  # Convertir de vuelta a escala de grises
    return shadowed_image

# Función para aplicar aumentos aleatorios y ajustar el tamaño de la imagen
def random_augment(image, steering_angle):
    image = convert_to_grayscale(image)  # Convertir la imagen a escala de grises
    if np.random.rand() < 0.5:
        image = apply_pan(image)
    if np.random.rand() < 0.5:
        image = apply_zoom(image)
    if np.random.rand() < 0.5:
        image = apply_random_brightness(image)
    if np.random.rand() < 0.5:
        image = apply_image_brighten(image)
    if np.random.rand() < 0.5:
        image = apply_random_shadow(image)
    if np.random.rand() < 0.5:
        image, steering_angle = random_flip(image, steering_angle)
    return image, steering_angle

6. **Funciones de Preprocesamiento de Imágenes y Codificación de Impulsos**: Estas funciones preprocesan las imágenes para convertirlas en un formato adecuado para la codificación en impulsos, normalizan los ángulos de dirección y realizan la codificación delta utilizando la librería snntorch

In [7]:
# Función para preprocesar la imagen para el entrenamiento del modelo CSNN
def img_preprocess(img):
    img = img[40:140, :]  # Recortar la imagen para eliminar características innecesarias
    if len(img.shape) == 3:  # Verificar si la imagen tiene 3 canales
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # Convertir a escala de grises
    img = cv2.GaussianBlur(img, (3, 3), 0)  # Aplicar desenfoque gaussiano
    img = cv2.resize(img, (160, 50), interpolation=cv2.INTER_AREA)  # Reducir el tamaño para facilitar el procesamiento
    img = (img - 128.) / 128.  # Normalizar los valores de los píxeles
    return img

# Función para convertir un valor a tensor
def value_to_tensor(value, tensor_length=21):
    value = round(value, 1)
    if value < -1 or value > 1:
        raise ValueError("El valor debe estar en el rango [-1, 1]")
    position = int(((value + 1) / 2) * (tensor_length - 1))
    tensor = torch.zeros(tensor_length, dtype=torch.int32)
    tensor[position] = 1
    return tensor

# Función para asegurar que el ángulo esté en el rango [-1, 1]
def clip_steering_angle(angle):
    return np.clip(angle, -1, 1)

# Codificación Delta usando snnTorch
def encode_delta(image1, image2):
    if not isinstance(image1, torch.Tensor):
        image1 = torch.tensor(image1, dtype=torch.float32)
    if not isinstance(image2, torch.Tensor):
        image2 = torch.tensor(image2, dtype=torch.float32)
    delta_encoded = spikegen.delta(image1, image2)
    return delta_encoded

# Función para preparar y codificar los datos de entrenamiento
def prepare_and_encode_train_data(image_paths, steering_angles, datadir, device='cuda'):
    positions = []
    delta_images = []
    angles = []

    # Ajustar el límite para asegurar que tengamos un número par de imágenes
    limit = len(image_paths) - (len(image_paths) % 2)

    for i in range(0, limit - 1, 2):
        image_path1, image_path2 = image_paths[i], image_paths[i+1]
        steering_angle2 = clip_steering_angle(steering_angles[i+1])

        full_image_path1 = os.path.join(datadir, image_path1)
        full_image_path2 = os.path.join(datadir, image_path2)

        image1 = mpimg.imread(full_image_path1)
        image2 = mpimg.imread(full_image_path2)

        image1, _ = random_augment(image1, clip_steering_angle(steering_angles[i]))
        image2, steering_angle2 = random_augment(image2, steering_angle2)

        image1 = img_preprocess(image1)
        image2 = img_preprocess(image2)

        image1_tensor = torch.tensor(image1, dtype=torch.float32, device=device)
        image2_tensor = torch.tensor(image2, dtype=torch.float32, device=device)

        # Asegurarse de que la codificación delta se aplica correctamente
        delta_image = encode_delta(image1_tensor, image2_tensor)

        # Añadir datos a las listas
        positions.append(i // 2)
        delta_images.append(delta_image)
        angles.append(value_to_tensor(steering_angle2).to(device))

    positions_tensor = torch.tensor(positions, dtype=torch.int32, device=device)
    delta_images_tensor = torch.stack(delta_images)
    angles_tensor = torch.stack(angles)

    return positions_tensor, delta_images_tensor, angles_tensor

# Función para preparar y codificar los datos de validación (sin aumentos)
def prepare_and_encode_valid_data(image_paths, steering_angles, datadir, device='cuda'):
    positions = []
    delta_images = []
    angles = []

    # Ajustar el límite para asegurar que tengamos un número par de imágenes
    limit = len(image_paths) - (len(image_paths) % 2)

    for i in range(0, limit - 1, 2):
        image_path1, image_path2 = image_paths[i], image_paths[i+1]
        steering_angle2 = clip_steering_angle(steering_angles[i+1])

        full_image_path1 = os.path.join(datadir, image_path1)
        full_image_path2 = os.path.join(datadir, image_path2)

        image1 = mpimg.imread(full_image_path1)
        image2 = mpimg.imread(full_image_path2)

        image1 = img_preprocess(image1)
        image2 = img_preprocess(image2)

        image1_tensor = torch.tensor(image1, dtype=torch.float32, device=device)
        image2_tensor = torch.tensor(image2, dtype=torch.float32, device=device)

        # Asegurarse de que la codificación delta se aplica correctamente
        delta_image = encode_delta(image1_tensor, image2_tensor)

        # Añadir datos a las listas
        positions.append(i // 2)
        delta_images.append(delta_image)
        angles.append(value_to_tensor(steering_angle2).to(device))

    positions_tensor = torch.tensor(positions, dtype=torch.int32, device=device)
    delta_images_tensor = torch.stack(delta_images)
    angles_tensor = torch.stack(angles)

    return positions_tensor, delta_images_tensor, angles_tensor

7. **Carga y Balanceo de Datos de Conducción**: Cargamos y balanceamos los datos de conducción para crear conjuntos de entrenamiento y validación.

In [8]:
# Cargar los datos de entrenamiento y validación
csv_file_path = '/content/TrainImagesTFM_1_2/driving_log.csv'
driving_data = load_driving_data(csv_file_path)
driving_data = simplify_image_path(driving_data, ['center_image', 'left_image', 'right_image'])
turn_counts = categorize_turns(driving_data, 'steering_angle')
balanced_driving_data = balance_data_by_replication(driving_data, 'turn_category')
percentile = 90
num_bins = 25
max_samples_per_bin = calculate_samples_per_bin(balanced_driving_data, 'steering_angle', num_bins, percentile)
balanced_data_adjusted = remove_samples_by_category(balanced_driving_data, samples_per_bin=max_samples_per_bin)
image_paths, steerings, X_train, X_valid, y_train, y_valid = trainValSample("/content/TrainImagesTFM_1_2/", balanced_data_adjusted)

Training Samples: 44179
Validation Samples: 11045
Total Samples: 55224


8. **Codificación de Datos y Guardado de Resultados**: Iniciamos el tracker de CodeCarbon para medir las emisiones de carbono durante el proceso de codificación, preparamos y codificamos los datos de entrenamiento y validación, y guardamos los resultados en archivos HDF5. Finalmente, copiamos los archivos a Google Drive.

In [9]:
# Dado que X_train, y_train, X_valid, y_valid están definidos
datadir = "/content/TrainImagesTFM_1_2/"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Capturar información del tiempo para guardado
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')

# Iniciar el tracker de CodeCarbon
tracker = EmissionsTracker(output_dir='/content/drive/MyDrive/UOC/TFM/EcoEfficiency_Reports/', project_name=f"emissions_{current_time}.csv")
tracker.start()

# Procesar los conjuntos de datos de entrenamiento y validación
train_positions, train_delta_images, train_angles = prepare_and_encode_train_data(X_train, y_train, datadir, device)
valid_positions, valid_delta_images, valid_angles = prepare_and_encode_valid_data(X_valid, y_valid, datadir, device)

# Verificar las dimensiones de los tensores generados
print(f"Dimensiones del tensor de posiciones de entrenamiento: {train_positions.shape}")
print(f"Dimensiones del tensor de imágenes delta de entrenamiento: {train_delta_images.shape}")
print(f"Dimensiones del tensor de ángulos de entrenamiento: {train_angles.shape}")

print(f"Dimensiones del tensor de posiciones de validación: {valid_positions.shape}")
print(f"Dimensiones del tensor de imágenes delta de validación: {valid_delta_images.shape}")
print(f"Dimensiones del tensor de ángulos de validación: {valid_angles.shape}")

# Guardar en formato HDF5
def save_to_hdf5(positions, delta_images, angles, file_name):
    with h5py.File(file_name, 'w') as f:
        f.create_dataset('positions', data=positions.cpu().numpy(), compression="gzip")
        f.create_dataset('delta_images', data=delta_images.cpu().numpy(), compression="gzip")
        f.create_dataset('angles', data=angles.cpu().numpy(), compression="gzip")

save_to_hdf5(train_positions, train_delta_images, train_angles, f'/content/training_data_pairs_{current_time}.hdf5')
save_to_hdf5(valid_positions, valid_delta_images, valid_angles, f'/content/validation_data_pairs_{current_time}.hdf5')

tracker.stop()

# Copiar archivos a Google Drive
shutil.copy2(f'/content/training_data_pairs_{current_time}.hdf5', f'/content/drive/MyDrive/UOC/TFM/Spike Encoding/Version_10/training_data_pairs_{current_time}.hdf5')
shutil.copy2(f'/content/validation_data_pairs_{current_time}.hdf5', f'/content/drive/MyDrive/UOC/TFM/Spike Encoding/Version_10/validation_data_pairs_{current_time}.hdf5')

print(f"Total de pares de imágenes codificadas y combinadas en entrenamiento: {train_positions.shape[0]}")
print(f"Total de pares de imágenes codificadas y combinadas en validación: {valid_positions.shape[0]}")


[codecarbon INFO @ 19:05:39] [setup] RAM Tracking...
[codecarbon INFO @ 19:05:39] [setup] GPU Tracking...
[codecarbon INFO @ 19:05:39] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 19:05:39] [setup] CPU Tracking...
[codecarbon WARNING @ 19:05:39] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 19:05:40] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 19:05:40] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 19:05:40] >>> Tracker's metadata:
[codecarbon INFO @ 19:05:40]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 19:05:40]   Python version: 3.10.12
[codecarbon INFO @ 19:05:40]   CodeCarbon version: 2.5.0
[codecarbon INFO @ 19:05:40]   Available RAM : 12.675 GB
[codecarbon INFO @ 19:05:40]   CPU count: 2
[codecarbon INFO @ 19:05:40]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 19:05:40]   GPU c

Dimensiones del tensor de posiciones de entrenamiento: torch.Size([22089])
Dimensiones del tensor de imágenes delta de entrenamiento: torch.Size([22089, 50, 160])
Dimensiones del tensor de ángulos de entrenamiento: torch.Size([22089, 21])
Dimensiones del tensor de posiciones de validación: torch.Size([5522])
Dimensiones del tensor de imágenes delta de validación: torch.Size([5522, 50, 160])
Dimensiones del tensor de ángulos de validación: torch.Size([5522, 21])


[codecarbon INFO @ 19:08:56] Energy consumed for RAM : 0.000257 kWh. RAM Power : 4.753040313720703 W
[codecarbon INFO @ 19:08:56] Energy consumed for all GPUs : 0.001601 kWh. Total GPU Power : 30.079081764785837 W
[codecarbon INFO @ 19:08:56] Energy consumed for all CPUs : 0.002302 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:08:56] 0.004160 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:08:59] Energy consumed for RAM : 0.000260 kWh. RAM Power : 4.753040313720703 W
[codecarbon INFO @ 19:08:59] Energy consumed for all GPUs : 0.001622 kWh. Total GPU Power : 30.504508883385085 W
[codecarbon INFO @ 19:08:59] Energy consumed for all CPUs : 0.002330 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:08:59] 0.004212 kWh of electricity used since the beginning.


Total de pares de imágenes codificadas y combinadas en entrenamiento: 22089
Total de pares de imágenes codificadas y combinadas en validación: 5522
